In [131]:
from vispy import app, scene
from vispy.scene.visuals import Text, Rectangle, Compound, Image
import napari
import numpy as np

viewer = napari.Viewer()
viewer.close()

In [299]:
from vispy import app, scene
from vispy.scene.visuals import Mesh
from vispy.color import Color
import numpy as np
from typing import Union

class MultiRectVisual(Mesh):
    def __init__(self, x: list, y: list, w: list, h: list, color: Union[list,str] = 'white', anchor_x='center', anchor_y='center', **kwargs):
        # convert color to list if it is a single color
        if isinstance(color, str):
            color = [color]
        # padd shape to match length
        max_len = max(len(x), len(y), len(w), len(h), len(color))
        # raise if any of the lists are too short
        if len(x) < max_len or len(y) < max_len or len(w) < max_len or len(h) < max_len:
            raise ValueError('x, y, w and h must all be the same length')
        color.extend(color[:1] * (max_len - len(color)))
        self._check_valid('anchor_x', anchor_x, ('left', 'center', 'right'))
        self._check_valid('anchor_y', anchor_y, ('top', 'center', 'middle', 'baseline', 'bottom'))

        self.anchor_x = anchor_x
        self.anchor_y = anchor_y

        self.rect_data = list(zip(x, y, w, h, color))
        vertices, faces, colors = self._generate_vertices_faces_and_colors()
        super(MultiRectVisual, self).__init__(vertices=vertices, faces=faces, vertex_colors=colors, **kwargs)

    def _generate_vertices_faces_and_colors(self):
        vertices = []
        faces = []
        colors = []
        face_index = 0

        for rect in self.rect_data:
            x, y, w, h, color_name = rect
            # Adjust x and y based on anchor
            x_offset, y_offset = self._calculate_anchor_offset(w, h)
            rect_vertices = np.array([
                [x - x_offset, y - y_offset], 
                [x - x_offset + w, y - y_offset], 
                [x - x_offset + w, y - y_offset + h], 
                [x - x_offset, y - y_offset + h]
            ])
            vertices.extend(rect_vertices)

            rect_faces = np.array([
                [face_index, face_index + 1, face_index + 2],
                [face_index, face_index + 2, face_index + 3]
            ])
            faces.extend(rect_faces)
            face_index += 4

            color = Color(color_name).rgba
            colors.extend([color] * 4)

        return np.array(vertices), np.array(faces), np.array(colors)
    
    def _calculate_anchor_offset(self, width, height):
        dx = dy = 0

        # Calculate the horizontal offset
        if self.anchor_x == 'right':
            dx = +width
        elif self.anchor_x == 'center':
            dx = +width/ 2

        # Calculate the vertical offset
        # Assuming that for rectangles, 'top' and 'bottom' relate to the actual top and bottom edges
        if self.anchor_y in ('top'):
            dy = +height * 1.1
        elif self.anchor_y in ('center', 'middle'):
            dy = +height *1.1 / 2
        elif self.anchor_y in ('baseline', 'bottom'):
            dy = +height * 0.1 / 2

        return dx, dy
    
    def _check_valid(self, name, value, valid_values):
        if value not in valid_values:
            raise ValueError(f"{name} must be one of {valid_values}, but got {value}")


    def update_rects(self, x: list, y: list, w: list, h: list, color: Union[list,str]):
        # convert color to list if it is a single color
        if isinstance(color, str):
            color = [color]
        # padd shape to match length
        max_len = max(len(x), len(y), len(w), len(h), len(color))
        # raise if any of the lists are too short
        if len(x) < max_len or len(y) < max_len or len(w) < max_len or len(h) < max_len:
            raise ValueError('x, y, w and h must all be the same length')
        color.extend(color[:1] * (max_len - len(color)))
        self.rect_data = list(zip(x, y, w, h, color))
        vertices, faces, colors = self._generate_vertices_faces_and_colors()
        self.set_data(vertices=vertices, faces=faces, vertex_colors=colors)

    @property
    def x(self):
        return [rect[0] for rect in self.rect_data]
    
    @x.setter
    def x(self, x: list):
        self.update_rects(x, self.y, self.w, self.h, self.color)

    @property
    def y(self):
        return [rect[1] for rect in self.rect_data]
    
    @y.setter
    def y(self, y: list):
        self.update_rects(self.x, y, self.w, self.h, self.color)

    @property
    def w(self):
        return [rect[2] for rect in self.rect_data]
    
    @w.setter
    def w(self, w: list):
        self.update_rects(self.x, self.y, w, self.h, self.color)

    @property
    def h(self):
        return [rect[3] for rect in self.rect_data]
    
    @h.setter
    def h(self, h: list):
        self.update_rects(self.x, self.y, self.w, h, self.color)

    @property
    def color(self):
        return [rect[4] for rect in self.rect_data]
    
    @color.setter
    def color(self, color: Union[list,str]):
        self.update_rects(self.x, self.y, self.w, self.h, color)

    @property
    def anchors(self):
        return self.anchor_x, self.anchor_y
    
    @anchors.setter
    def anchors(self, anchors: tuple):
        self.anchor_x, self.anchor_y = anchors
        self.update_rects(self.x, self.y, self.w, self.h, self.color)

In [300]:
from vispy import app, scene
from vispy.scene.visuals import Mesh
from vispy.color import Color
import numpy as np
from typing import Union

class MultiRectVisual(Mesh):
    def __init__(self, x: list, y: list, w: list, h: list, color: Union[list,str] = 'white', anchor_x='center', anchor_y='center', **kwargs):
        # convert color to list if it is a single color
        if isinstance(color, str):
            color = [color]
        # padd shape to match length
        max_len = max(len(x), len(y), len(w), len(h), len(color))
        # raise if any of the lists are too short
        if len(x) < max_len or len(y) < max_len or len(w) < max_len or len(h) < max_len:
            raise ValueError('x, y, w and h must all be the same length')
        color.extend(color[:1] * (max_len - len(color)))
        self._check_valid('anchor_x', anchor_x, ('left', 'center', 'right'))
        self._check_valid('anchor_y', anchor_y, ('top', 'center', 'middle', 'baseline', 'bottom'))

        self.anchor_x = anchor_x
        self.anchor_y = anchor_y

        self.rect_data = list(zip(x, y, w, h, color))
        vertices, faces, colors = self._generate_vertices_faces_and_colors()
        super(MultiRectVisual, self).__init__(vertices=vertices, faces=faces, vertex_colors=colors, **kwargs)

    def _generate_vertices_faces_and_colors(self):
        vertices = []
        faces = []
        colors = []
        face_index = 0

        for rect in self.rect_data:
            x, y, w, h, color_name = rect
            # Adjust x and y based on anchor
            x_offset, y_offset = self._calculate_anchor_offset(w, h)
            rect_vertices = np.array([
                [x - x_offset, y - y_offset], 
                [x - x_offset + w, y - y_offset], 
                [x - x_offset + w, y - y_offset + h], 
                [x - x_offset, y - y_offset + h]
            ])
            vertices.extend(rect_vertices)

            rect_faces = np.array([
                [face_index, face_index + 1, face_index + 2],
                [face_index, face_index + 2, face_index + 3]
            ])
            faces.extend(rect_faces)
            face_index += 4

            color = Color(color_name).rgba
            colors.extend([color] * 4)

        return np.array(vertices), np.array(faces), np.array(colors)
    
    def _calculate_anchor_offset(self, width, height):
        dx = dy = 0

        # Calculate the horizontal offset
        if self.anchor_x == 'right':
            dx = +width
        elif self.anchor_x == 'center':
            dx = +width/ 2

        # Calculate the vertical offset
        # Assuming that for rectangles, 'top' and 'bottom' relate to the actual top and bottom edges
        if self.anchor_y in ('top'):
            dy = +height * 1.1
        elif self.anchor_y in ('center', 'middle'):
            dy = +height *1.1 / 2
        elif self.anchor_y in ('baseline', 'bottom'):
            dy = +height * 0.1 / 2

        return dx, dy
    
    def _check_valid(self, name, value, valid_values):
        if value not in valid_values:
            raise ValueError(f"{name} must be one of {valid_values}, but got {value}")


    def update_rects(self, x: list, y: list, w: list, h: list, color: Union[list,str]):
        # convert color to list if it is a single color
        if isinstance(color, str):
            color = [color]
        # padd shape to match length
        max_len = max(len(x), len(y), len(w), len(h), len(color))
        # raise if any of the lists are too short
        if len(x) < max_len or len(y) < max_len or len(w) < max_len or len(h) < max_len:
            raise ValueError('x, y, w and h must all be the same length')
        color.extend(color[:1] * (max_len - len(color)))
        self.rect_data = list(zip(x, y, w, h, color))
        vertices, faces, colors = self._generate_vertices_faces_and_colors()
        self.set_data(vertices=vertices, faces=faces, vertex_colors=colors)

    @property
    def x(self):
        return [rect[0] for rect in self.rect_data]
    
    @x.setter
    def x(self, x: list):
        self.update_rects(x, self.y, self.w, self.h, self.color)

    @property
    def y(self):
        return [rect[1] for rect in self.rect_data]
    
    @y.setter
    def y(self, y: list):
        self.update_rects(self.x, y, self.w, self.h, self.color)

    @property
    def w(self):
        return [rect[2] for rect in self.rect_data]
    
    @w.setter
    def w(self, w: list):
        self.update_rects(self.x, self.y, w, self.h, self.color)

    @property
    def h(self):
        return [rect[3] for rect in self.rect_data]
    
    @h.setter
    def h(self, h: list):
        self.update_rects(self.x, self.y, self.w, h, self.color)

    @property
    def color(self):
        return [rect[4] for rect in self.rect_data]
    
    @color.setter
    def color(self, color: Union[list,str]):
        self.update_rects(self.x, self.y, self.w, self.h, color)

    @property
    def anchors(self):
        return self.anchor_x, self.anchor_y
    
    @anchors.setter
    def anchors(self, anchors: tuple):
        self.anchor_x, self.anchor_y = anchors
        self.update_rects(self.x, self.y, self.w, self.h, self.color)

class TextWithBoxVisual(Compound):
    def __init__(self, text: Union[list[str], str], color: Union[list[str], str] = 'white', bgcolor: str = 'red', font_size: int = 12, pos: Union[list[tuple], tuple] = (0, 0), anchor_x: str = 'left', anchor_y: str = 'bottom', bold=False, italic=False, **kwargs):
        self._textvisual = Text(text=text, pos=pos, color=color, font_size=font_size, anchor_x=anchor_x, anchor_y=anchor_y, bold=bold, italic=italic)
        # bring arguments to correct shape
        if isinstance(pos, tuple):
            pos_x = [pos[0]]
            pos_y = [pos[1]]
        else:
            pos_x = [p[0] for p in pos]
            pos_y = [p[1] for p in pos]

        self._rectagles_visual = MultiRectVisual(
            x=pos_x,
            y=pos_y,
            w=[font_size*1.5]*len(pos_x),
            h=[font_size*1.5]*len(pos_y),
            color=bgcolor,
            anchor_x=anchor_x,
            anchor_y=anchor_y,
        )

        super(TextWithBoxVisual, self).__init__([self._rectagles_visual, self._textvisual])
   
    @property
    def color(self):
        return self._textvisual.color
    
    @color.setter
    def color(self, color: str):
        self._textvisual.color = color

    @property
    def bgcolor(self):
        return self._rectagles_visual.color
    
    @bgcolor.setter
    def bgcolor(self, color: str):
        self._rectagles_visual.color = color

    @property
    def font_size(self):
        return self._textvisual.font_size
    
    @font_size.setter
    def font_size(self, size: int):
        self._textvisual.font_size = size
        self._rectagles_visual.h = [size*1.5]

    @property
    def pos(self):
        return self._textvisual.pos
    
    @pos.setter
    def pos(self, pos: tuple):
        self._textvisual.pos = pos
        if isinstance(pos, tuple):
            pos_x = [pos[0]]
            pos_y = [pos[1]]
        else:
            pos_x = [p[0] for p in pos]
            pos_y = [p[1] for p in pos]
        self._rectagles_visual.x = pos_x
        self._rectagles_visual.y = pos_y

    
    @property
    def text(self):
        return self._textvisual.text
    
    @text.setter
    def text(self, text: Union[list[str], str]):
        self._textvisual.text = text

    @property
    def anchors(self):
        return self._textvisual.anchors
    
    @anchors.setter
    def anchors(self, anchors: tuple):
        self._textvisual.anchors = anchors
        self._rectagles_visual.anchors = anchors

    def update_data(self, text: Union[list[str], str], color: Union[list[str], str] = 'white', bgcolor: str = 'red', font_size: int = 12, pos: Union[list[tuple], tuple] = (0, 0)):
        # bring arguments to correct shape  
        if isinstance(pos, tuple):
            pos_x = [pos[0]]
            pos_y = [pos[1]]
        else:
            pos_x = [p[0] for p in pos]
            pos_y = [p[1] for p in pos]
        self._textvisual.text = text
        self._textvisual.color = color
        self._textvisual.font_size = font_size
        self._textvisual.pos = pos
        self._rectagles_visual.update_rects(pos_x, pos_y, [font_size*1.2]*len(pos_x), [font_size*1.2]*len(pos_y), bgcolor)

In [301]:
canvas = scene.SceneCanvas(keys='interactive', size=(600, 400), show=True)
view = canvas.central_widget.add_view()

multi_rect_visual = TextWithBoxVisual(text='Hello World', color='white', bgcolor='red', font_size=24, pos=(100, 100))
view.add(multi_rect_visual)

if __name__ == '__main__':
    app.run()

In [302]:
multi_rect_visual.font_size = 48

In [245]:
multi_rect_visual.update_data(text=['Hello World', 'Now', 'its', 'your turn', "really"], color='white', bgcolor='green', font_size=24, pos=[(100, 100), (150, 200), (250, 350), (350, 100), (50, 50)])

In [303]:
multi_rect_visual.pos = (200,200)

In [304]:
multi_rect_visual._rectagles_visual.w = [400]

In [311]:
multi_rect_visual.anchors = ("left", "bottom")

In [312]:
multi_rect_visual.anchors = ("right", "bottom")

In [313]:
multi_rect_visual.anchors = ("center", "bottom")

In [314]:
multi_rect_visual.anchors = ("left", "top")

In [315]:
multi_rect_visual.anchors = ("center", "top")

In [316]:
multi_rect_visual.anchors = ("right", "top")

In [319]:
multi_rect_visual.bgcolor = 'white'